In [1]:
from pyvi import ViTokenizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

from va.src.utils.data import load_va_data

## Load Data

In [3]:
PATH = "../data/trungquan/trainset.xlsx"
X, y = load_va_data(PATH)

## Text Embedding

### TF-IDF

In [8]:
tfidf = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")